# Import necessary packages

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 3"
import tasks
import utils
from rouge_score import rouge_scorer
import random
import numpy as np
import torch
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from datasets import load_dataset
import json
from tqdm import tqdm

/home/lishuo1/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up indexer

In [2]:
torch.set_grad_enabled(False)
from haystack.nodes import DensePassageRetriever
from haystack.document_stores.faiss import FAISSDocumentStore
document_store_faiss = FAISSDocumentStore(faiss_index_path="bio_faiss_index.faiss")
reloaded_retriever = DensePassageRetriever.load(load_dir='dpr_bio', document_store=document_store_faiss)

/home/lishuo1/anaconda3/envs/glu/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
task='bio'
questions, contexts, answers = tasks.bio_dpr(task=task).load_dataset()

In [4]:
def hit_rate(ctx_ids_list, retrieved_ids_list):
    rate = []
    for ctx_ids, retrieved_ids in zip(ctx_ids_list, retrieved_ids_list):
        # check if any id in true ids in retrieved ids
        if any([ctx_id in retrieved_ids for ctx_id in ctx_ids]):
            rate.append(True)
        else:
            rate.append(False)
    return np.mean(rate)

In [7]:
ctx_ids_list = []
retrieved_ids_list = []
ctx_ids_list = [[tmp['id'] for tmp in c] for c in contexts]
retrieved_docs = reloaded_retriever.retrieve_batch(questions, top_k=20)
retrieved_ids_list = [[d.id for d in docs] for docs in retrieved_docs]
rate = hit_rate(ctx_ids_list, retrieved_ids_list)
print("hit rate: ", rate)

hit rate:  0.8002822865208187


In [15]:
semantic = False
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)
if semantic:
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    # setup semantic model
    semantic_tokenizer = \
        AutoTokenizer.from_pretrained("microsoft/deberta-large-mnli")
    semantic_model = \
        AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-large-mnli"
        ).cuda()

# Collect data

In [16]:
indices = np.arange(len(queries))
random.shuffle(indices)
cal_indices = indices[:int(len(indices) * 0.5)]
test_indices = indices[int(len(indices) * 0.5):]

In [ ]:
question_embedding = q_encoder(**q_tokenizer(query, return_tensors="pt", padding=True))
question_embedding = question_embedding[0].numpy()
scores, retrieved_examples = wiki.get_nearest_examples_batch('embeddings', question_embedding, k=20)

In [17]:
import pickle
def write_list(a_list, file_name):
    # store list in binary file so 'wb' mode
    with open(file_name, 'wb') as fp:
        pickle.dump(a_list, fp)
#         print('Done writing list into a binary file')
def read_list(file_name):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

def save_results(task):
    # save retrieved_scores to a pickle file
    write_list(retrieved_scores, f'chatgpt_retrieved_scores_{task}.p')
    # save retrieved_true_scores to a pickle file
    write_list(retrieved_true_scores, f'chatgpt_retrieved_true_scores_{task}.p')
    # save queries to a pickle file
    write_list(queries, f'chatgpt_queries_{task}.p')
    # save answers to a pickle file
    write_list(answers, f'chatgpt_true_answers_{task}.p')
    # save passages to a pickle file
    write_list(passages, f'chatgpt_passages_{task}.p')
    # save chatgpt_true_scores to a pickle file
    write_list(chatgpt_true_scores, f'chatgpt_true_scores_{task}.p')
    # save chatgpt_texts to a pickle file
#     write_list(chatgpt_texts, f'chatgpt_texts_{task}.p')
    # save chatgpt_answers to a pickle file
    write_list(chatgpt_answers, f'chatgpt_answers_{task}.p')
    # save chatgpt_semantics to a picle file
    write_list(chatgpt_semantics, f'chatgpt_semantics_{task}.p')
    # save feasibilities to a pickle file
    write_list(feasibilities, f'chatgpt_feasibilities_{task}.p')
    # save occurances to a pickle file
    write_list(occurances, f'chatgpt_occurances_{task}.p')
    # save semantic_ids to a pickle file
    write_list(semantic_ids, f'chatgpt_semantic_ids_{task}.p')
    # save probs to a picle file
    write_list(probs, f'chatgpt_probs_{task}.p')
    
    write_list(retrieved_scores_unc, f'chatgpt_retrieved_scores_unc_{task}.p')
    write_list(retrieved_true_scores_unc, f'chatgpt_retrieved_true_scores_unc_{task}.p')
    write_list(queries_unc, f'chatgpt_queries_unc_{task}.p')
    write_list(answers_unc, f'chatgpt_answers_unc_{task}.p')
    write_list(passages_unc, f'chatgpt_passages_unc_{task}.p')
    write_list(chatgpt_true_scores_unc, f'chatgpt_true_scores_unc_{task}.p')
    write_list(chatgpt_answers_unc, f'chatgpt_answers_unc_{task}.p')
    write_list(occurances_unc, f'chatgpt_occurances_unc_{task}.p')
    write_list(semantic_ids_unc, f'chatgpt_semantic_ids_unc_{task}.p')
    write_list(probs_unc, f'chatgpt_probs_unc_{task}.p')

In [18]:
def read_results(task):
    retrieved_scores = read_list(f'chatgpt_retrieved_scores_{task}.p')
    retrieved_true_scores = read_list(f'chatgpt_retrieved_true_scores_{task}.p')
    queries = read_list(f'chatgpt_queries_{task}.p')
    answers = read_list(f'chatgpt_answers_{task}.p')
    chatgpt_true_scores = read_list(f'chatgpt_true_scores_{task}.p')
    chatgpt_answers = read_list(f'chatgpt_answers_{task}.p')
    chatgpt_passages = read_list(f'chatgpt_passages_{task}.p')
    chatgpt_semantics = read_list(f'chatgpt_semantics_{task}.p')
    chatgpt_occurances = read_list(f'chatgpt_occurances_{task}.p')
    chatgpt_semantic_ids = read_list(f'chatgpt_semantic_ids_{task}.p')
    chatgpt_probs = read_list(f'chatgpt_probs_{task}.p')
    
    retrieved_scores_unc = read_list(f'chatgpt_retrieved_scores_unc_{task}.p')
    retrieved_true_scores_unc = read_list(f'chatgpt_retrieved_true_scores_unc_{task}.p')
    queries_unc = read_list(f'chatgpt_queries_unc_{task}.p')
    answers_unc = read_list(f'chatgpt_answers_unc_{task}.p')
    passages_unc = read_list(f'chatgpt_passages_unc_{task}.p')
    chatgpt_true_scores_unc = read_list(f'chatgpt_true_scores_unc_{task}.p')
    chatgpt_answers_unc = read_list(f'chatgpt_answers_unc_{task}.p')
    chatgpt_occurances_unc = read_list(f'chatgpt_occurances_unc_{task}.p')
    chatgpt_semantic_ids_unc = read_list(f'chatgpt_semantic_ids_unc_{task}.p')
    chatgpt_probs_unc = read_list(f'chatgpt_probs_unc_{task}.p')
    
    return retrieved_scores, retrieved_true_scores, queries, answers, chatgpt_true_scores, chatgpt_answers, chatgpt_passages, chatgpt_semantics, chatgpt_occurances, chatgpt_semantic_ids, chatgpt_probs, retrieved_scores_unc, retrieved_true_scores_unc, queries_unc, answers_unc, passages_unc, chatgpt_true_scores_unc, chatgpt_answers_unc, chatgpt_occurances_unc, chatgpt_semantic_ids_unc, chatgpt_probs_unc

## Setup chatgpt

In [19]:
utils.setup_openai()

In [60]:
%%time
chat = True
semantic = False
queries = []
# answers = []
passages = []
retrieved_scores = []
retrieved_true_scores = []
chatgpt_true_scores = []
chatgpt_texts = []
chatgpt_answers = []
chatgpt_semantics = []
semantic_probs = []
feasibilities = []
occurances = []
semantic_ids = []
probs = []
input_token_counts = []
output_token_counts = []

retrieved_scores_unc = []
retrieved_true_scores_unc = []
queries_unc = []
answers_unc = []
passages_unc = []
chatgpt_true_scores_unc = []
chatgpt_answers_unc = []
occurances_unc = []
semantic_ids_unc = []
probs_unc = []
        
# for idx, (element, score, retrieved) in enumerate(zip(elements, scores, retrieved_examples)):
for idx, (question, context, answer) in enumerate(zip(questions, contexts, answers)):
    if len(queries) > 30:
        break
    print(idx)
    # print(f'{idx}', file=open(f'chatgpt_{task}.txt', 'a'))
    # feasible = False
    # if idx % 10 == 0:x
    #     save_results(task)
    passage_id = [c['id'] for c in context]
    # passage_title = [c['title'] for c in context]
    passage_text = [c['text'] for c in context]

    # query, answer, passage_id, passage_title, passage_text = \
    #     dataset_info(element, dataset=task)
    # if len(passage_id) == 0:
    #     continue
    # retrieved_ids, retrieved_texts, retrieved_title, true_score = \
    #     utils.retrieved_info(score, retrieved, passage_id[0])
    # if len(true_score) == 0:
    #     continue
    
    prompt = utils.get_prompt_template(question, passage_text[0], task='Natural Questions')
    if chat:
        sequences, input_token_count, output_token_count = \
            utils.ask_chatgpt(prompt, n_answers=30, model="gpt-3.5-turbo-0613")
    else:
        sequences, probs = utils.ask_chatgpt(prompt)

    semantic_set_ids, semantic_probs, item_occurance = \
        utils.clustering(sequences, prompt, scorer=scorer)
    true_scores, matched_answer, semantics = utils.processing_answers(
        semantic_set_ids, semantic_probs, 
        item_occurance, answer, scorer,
        threshold=0.3
    )
    if len(true_scores) == 0:
        # retrieved_scores_unc.append(score)
        # retrieved_true_scores_unc.append(true_score)
        queries_unc.append(query)
        answers_unc.append(answer)
        passages_unc.append(passage_text)
        chatgpt_true_scores_unc.append(true_scores)
        chatgpt_answers_unc.append(sequences)
        occurances_unc.append(item_occurance)
        semantic_ids_unc.append(semantic_set_ids)
        probs_unc.append(semantic_probs)
        input_token_counts.append(input_token_count)
        output_token_counts.append(output_token_count)
    else:
        feasible = True
        # retrieved_scores.append(score)
        # retrieved_true_scores.append(true_score)
        queries.append(query)
        answers.append(answer)
        passages.append(passage_text)
        chatgpt_true_scores.append(true_scores)

        # probs_tmp = []
        # answers_tmp = []
        # semantic_id_tmp = []
        # occurance_tmp = []
        # semantic_tmp = []
        # for context, s in zip(retrieved_texts, score):

        #     prompt = utils.get_prompt_template(query, context, task='Natural Questions')
        #     if chat:
        #         sequences, input_token_count_tmp, output_token_count_tmp = \
        #             utils.ask_chatgpt(prompt, n_answers=30, model="gpt-3.5-turbo-0613")
        #     else:
        #         sequences, probs = utils.ask_chatgpt(prompt, n_answers=30)
        #     input_token_count += input_token_count_tmp
        #     output_token_count += output_token_count_tmp

        #     if semantic:
        #         semantic_set_ids, semantic_probs, item_occurance = \
        #             utils.compute_semantic_clusterring(
        #                 semantic_model, 
        #                 semantic_tokenizer,
        #                 prompt,
        #                 sequences,
        #             )
        #     else:
        #         semantic_set_ids, semantic_probs, item_occurance = \
        #             utils.clustering(sequences, prompt, scorer=scorer)

        #     probs_tmp.append(semantic_probs)
        #     answers_tmp.append(sequences)
        #     occurance_tmp.append(item_occurance)
        #     semantic_id_tmp.append(semantic_set_ids)

        # chatgpt_answers.append(answers_tmp)
        # feasibilities.append(feasible)
        # occurances.append(occurance_tmp)
        # semantic_ids.append(semantic_id_tmp)
        # probs.append(probs_tmp)
        # input_token_counts.append(input_token_count)
        # output_token_counts.append(output_token_count)
print('Finished!', file=open(f'chatgpt_{task}.txt', 'a'))

0


1
2
3
4
5
6
7


KeyboardInterrupt: 

In [61]:
len(queries)

5

## Estimated cost for each query

In [ ]:
average_cost = np.mean(input_token_counts) / 1000 * 0.0015 + \
               np.mean(output_token_counts) / 1000 * 0.002
print(average_cost)